In [1]:
import numpy as np
import RNTK_NEW
import jax
import symjax
import symjax.tensor as T
import math

In [2]:
L = 2
Ti = 2
TiPrime = 2
qmatrix = np.zeros([L,Ti],dtype = object)
DATA = T.Placeholder((Ti, 8), 'float32')
X = DATA[:,0]
tx = X*X[:, None]
n = tx.shape[0]
su = 1
sb = 1
sh = 1
sq = 1
sw = 1

In [3]:
def normv(x):
    return T.sqrt(T.sum(T.power(x, 2)))
def capQ(lq):
    return T.full((2,2), lq)

In [4]:
def VTq(M):
    A = T.diag(M)  # GP_old is in R^{n*n} having the output gp kernel
    # of all pairs of data in the data set
    B = A * A[:, None]
    C = T.sqrt(B)  # in R^{n*n}
    D = M / C  # this is lamblda in ReLU analyrucal formula
    E = T.clip(D, -1, 1)  # clipping E between -1 and 1 for numerical stability.
    F = (1 / (2 * np.pi)) * (E * (np.pi - T.arccos(E)) + T.sqrt(1 - E ** 2)) * C
    G = (np.pi - T.arccos(E)) / (2 * np.pi)
    return F,G

In [5]:
for t in range(0, Ti):
    for l in range(0, L):
        print(l, t)
        if ((t == l) & (l == 0)):
            #assuming ||x|| means the norm of that vector
            qmatrix[l, t] = su**2 * normv(X[0])**2 + sb**2 + sh ** 2

        if ((t == 0) & (l > t)):

            S,D = VTq(capQ(qmatrix[l-1,t]))
            qmatrix[l, t] = su**2 * S + sb**2

        if ((t > l) & (l == 0)):
            S,D = VTq(capQ(qmatrix[l,t-1]))
            qmatrix[l, t] = sw**2 * S + su**2 * normv(X[t])**2 + sb**2

        if ((t > l) & (l > 0)):
            S_left, D_left = VTq(capQ(qmatrix[l,t-1]))
            S_right, D_right = VTq(capQ(qmatrix[l-1,t]))
            qmatrix[l, t] = sw**2 * S_left + su**2 * D_right + sb**2


0 0
1 0
0 1
1 1


In [6]:
#MAKE ALGORITHM 2

In [7]:
# thetamatrix = np.zeros([Ti,TiPrime],dtype = object)
# kmatrix = np.zeros([Ti,TiPrime],dtype = object)
# for t in range(1, Ti):
#     for tp in range(1, TiPrime):
#         for l in range(1, L):
#             if (t == 1) & (tp == 1) & (l == 1):

In [8]:
len1 = 4
len2 = 4
for i in range(0,len1):
    for j in range(0,len2):
        print(i, j)

0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
3 0
3 1
3 2
3 3


In [19]:
switch_flag = 1
dim_1_i = dim_1 = 3
dim_2 = 3
dim_2_i = 0
dim_num = l1 + l2 + 1
print(dim_1_i, dim_2_i, switch_flag)
for d in range(0,dim_num):
    dim_1_idx = dim_1_i
    dim_2_idx = dim_2_i

    print("dim_1_i", dim_1_i, "dim_2_i", dim_2_i, switch_flag)
    while ((dim_1_idx <= dim_1) & (dim_2_idx <= dim_2)):
        print("inner iteration - ", dim_1_idx, dim_2_idx)
        dim_1_idx+=1
        dim_2_idx+=1

    if dim_1_i == 0:
        switch_flag -= 1
    else:
        dim_1_i = dim_1_i - 1
    if switch_flag <= 0:
        dim_2_i = dim_2_i + 1

3 0 1
dim_1_i 3 dim_2_i 0 1
inner iteration -  3 0
dim_1_i 2 dim_2_i 0 1
inner iteration -  2 0
inner iteration -  3 1
dim_1_i 1 dim_2_i 0 1
inner iteration -  1 0
inner iteration -  2 1
inner iteration -  3 2
dim_1_i 0 dim_2_i 0 1
inner iteration -  0 0
inner iteration -  1 1
inner iteration -  2 2
inner iteration -  3 3
dim_1_i 0 dim_2_i 1 0
inner iteration -  0 1
inner iteration -  1 2
inner iteration -  2 3
dim_1_i 0 dim_2_i 2 -1
inner iteration -  0 2
inner iteration -  1 3
dim_1_i 0 dim_2_i 3 -2
inner iteration -  0 3
